<a href="https://colab.research.google.com/github/31indianaSKKU/NBA_CareerLength/blob/main/NBA_Draft_History.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 이 부분은 처음 한번만 실행하면 됌.
# "The reason is that the last Ubuntu update update supports chromium driver just via snap."
# 최근 우분투 업데이트에서 크롬 드라이버 설치를 snap을 이용해서만 하도록 바뀜
# 고로 snap 없이 설치하는 아래 우회 코드로 변경
# 출처 : https://colab.research.google.com/drive/1cbEvuZOhkouYLda3RqiwtbM-o9hxGLyC
# 출처2 : https://stackoverflow.com/questions/75155063/selenium-use-chrome-on-colab-got-unexpectedly-exited

%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver


# Install selenium
pip install selenium

In [ ]:
# 셀레니움 module import 
# from selenium import webdriver
# 2023.05.08 selenium webdriver 사용방법 변경 
from selenium.webdriver.common.by import By
from selenium import webdriver
import time 
from time import sleep
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import requests
import re
import os
import csv

# Dataframe 
import pandas as pd

class Draft_history:
    def __init__(self, name, team, affiliation, year, round_num, pick_num, overall_num):
        self.name = name
        self.team = team
        self.affiliation = affiliation
        self.year = year
        self.round_num = round_num
        self.pick_num = pick_num
        self.overall_num = overall_num

    def to_list(self):
        return [self.name, self.team, self.affiliation, self.year, self.round_num, self.pick_num, self.overall_num]
##################################################################################################################################
##################################################################################################################################
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('window-size=1080,3840')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('disable-dev-shm-usage')
chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
browser = webdriver.Chrome('chromedriver', options=chrome_options)
##################################################################################################################################
##################################################################################################################################


for yrs in range(2015, 2017):
    browser.implicitly_wait(10)
    browser.get(f"https://www.nba.com/stats/draft/history?Season={yrs}")

    draft_list = []
    while True:
        try:
##################################################################################################################################
##################################################################################################################################
            browser.implicitly_wait(5)
            
            next_button = browser.find_elements(By.CLASS_NAME, 'Pagination_button__sqGoH')[1]

            tbody = browser.find_element(By.CLASS_NAME, 'Crom_body__UYOcU')
            trs = tbody.find_elements(By.TAG_NAME, 'tr')

            for tr in trs:
                tds = tr.find_elements(By.TAG_NAME, 'td')
                if len(tds) == 0:
                    continue
                else:
                    name = tds[0].text
                    team = tds[1].text
                    affiliation = tds[2].text
                    year = tds[3].text
                    round_num = tds[4].text
                    pick_num = tds[5].text
                    overall_num = tds[6].text

                    draft = Draft_history(name, team, affiliation, year, round_num, pick_num, overall_num)
                    draft_list.append(draft.to_list())

            has_next = next_button.is_enabled()
            if(has_next == False):
                print("end page")
                break

            browser.execute_script("arguments[0].click();", next_button)
            
        except Exception as e:
            print(e)

    # 파일에 데이터를 저장합니다.
    file_name = f'draft_list_{yrs}.csv'
    with open(file_name, 'w', encoding="utf-8", newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Name', 'Team', 'Affiliation', 'Year', 'Round', 'Pick', 'Overall'])
        
        for draft in draft_list:
            print(draft)
            writer.writerow(draft)

# Chrome 드라이버를 종료합니다.
browser.quit()
